### Foldy-Lax
The Foldy-Lax formulation provides a way of calculating the wavefield around isotropic scatterers. In the case of $N$ point scatterers only an $N \times N$ matrix equation needs to be solved. The wavefield, $\phi$, must satisy the Helmholtz equation in free space:

$$\nabla^2 \phi + k^2 \phi = 0,$$

where $k$ is the wavenumber. If the incident waves satisfy the Helmholtz equation, so must the scattered waves. 
Therefore when a plane wave $e^{i\vec{k}.\vec{r}}$ is incident upon a point scatterer at $\vec{r_1}$, the scattered wavefield, $\phi_s$, must be a solution to

$$\nabla^2 \phi_s + k^2 \phi_s = \delta(\vec{r} - \vec{r_1}) \sigma_1 \phi_1$$

since a point scatterer of strength $\sigma_1$ in a wavefield of local amplitude $\phi_1$ acts like a source $\delta(\vec{r} - \vec{r_1}) \sigma_1 \phi_1$. For multiple scatterers, we have an overall wavefield at $\vec{r}$ of

$$\phi(\vec{r}) = \phi_{inc}(\vec{r}) + \sum_j G(\vec{r}, \vec{r}_j) \sigma_j \phi_j,$$

where $G$ is the Green's function of the Helmholtz equation, the sum is over point scatterers, and the 2$^{nd}$ term on the right is the scattered component of the wavefield. In 3D, 

$$G(\vec{r}, \vec{r_i}) = \frac{1}{4\pi} \frac{e^{ik|\vec{r}-\vec{r_i}|}}{|\vec{r}-\vec{r_i}|}.$$

Now to get the matrix equation out from all of that, set $\vec{r} = \vec{r_i}$ and exclude the self-interaction at scatterer $i$:

$$\phi_i - \sum_{j \neq i} G_{ij} \sigma_j \phi_j = \phi_{inc}(\vec{r_i}),$$
or in matrix form

$$\sum_j M_{ij} \phi_i = \phi_{inc}(\vec{r_i}),$$

$$M_{ij} = \delta_{ij} + (\delta_{ij} - 1) \sigma_j G_{ij}.$$

This matrix equation can be solved for the wave amplitudes at each scatterer. The amplitude at $\vec{r}$ is then given by the sum of the incident, $\phi_{inc}$, and scattered, $G(\vec{r}, \vec{r}_j) \sigma_j \phi_j$, waves.

In [67]:
#import libraries
import numpy as np
from scipy import linalg as la
import matplotlib.pyplot as plt

#set the incident wavevector
k = [2.0, 0.0, 0.0]
#and wavenumber
K = la.norm(k)

#point scatter class
class Scatterer:
    def __init__(self, position, strength):
        self.r = np.array(position)
        self.s = strength
        self.phi = 0.0 #wave amplitude at scatterer
    
#Green's function for 3D Helmholtz
def green(r1, r2):
    return np.exp(1.0j*K*la.norm(r1-r2))/(4.0*np.pi*la.norm(r1-r2))

#incident wave amplitude
def incident_wave(r):
    return np.exp(1.0j*np.dot(k,r)) 

#scattered wave amplitude
#from a single scatterer
def scattered_wave(scatterer, r):
    return (scatterer.s)*(scatterer.phi)*green(r,scatterer.r)
    
#create a list of N point scatterers
N = 5
p = []
for i in range(N):
    position = np.array([4.0*(i-2), 0.0, 0.0])
    strength = 1.0
    p.append(Scatterer(position, strength))
    
#calculate the amplitude of the incident
#plane wave at each scatterer
incident_amplitudes = []
for i in range(N):
    incident_amplitudes.append(incident_wave(p[i].r))
    
#calculate the elements of the matrix
M = np.zeros((N,N), dtype = np.complex128)
for i in range(N):
    for j in range(N):
        if i == j:
            M[i][j] = 1.0
        else:
            M[i][j] = -(p[j].s)*green(p[i].r, p[j].r)

#solve for the wave amplitudes at each scatterer and
#assign these to the instances of the 'Scatterer' class
scatterer_amplitudes = la.solve(M, incident_amplitudes)
for i in range(N):
    p[i].phi = scatterer_amplitudes[i]
    
#calculate the total wave amplitude
#at all values of x and y
x = np.linspace(-20.0, 20.0, 100)
y = np.linspace(-20.0, 20.0, 100)
amplitudes = np.zeros((y.shape[0], x.shape[0]), dtype = np.complex128)
for i in range(y.shape[0]):
    for j in range(x.shape[0]):
        r = [x[j],y[i],0.0]
        amplitudes[i][j] = incident_wave(r)
        for scatterer in p:
            amplitudes[i][j] += scattered_wave(scatterer, r)

#plot the wave field
plt.pcolor(x,y,np.absolute(amplitudes), cmap = 'magma')
plt.title("Plane Waves Incident on Point Scatterers")
plt.title.text_font_size = 5
plt.xlabel("x")
plt.ylabel("y")
plt.colorbar()
plt.show()